In [ ]:
import os
import cv2
import math
import random
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def draw_bounding_boxes(image, annotations):
    for annotation in annotations:
        label, x_center, y_center, width, height = annotation
        x_center *= image.shape[1]
        y_center *= image.shape[0]
        width *= image.shape[1]
        height *= image.shape[0]
        x_min = int(x_center - width / 2)
        y_min = int(y_center - height / 2)
        x_max = int(x_center + width / 2)
        y_max = int(y_center + height / 2)
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

In [ ]:
def horizontal_flip(image, annotations):
    # Flip the image horizontally
    flipped_image = cv2.flip(image, 1)

    # Update bounding box coordinates
    flipped_annotations = []
    for annotation in annotations:
        label, x, y, width, height = annotation
        # Adjust the x-coordinate to reflect the horizontal flip
        x = 1 - x
        flipped_annotations.append((label, x, y, width, height))

    return flipped_image, flipped_annotations

In [ ]:
def vertical_flip(image, annotations):
    # Flip the image vertically
    flipped_image = cv2.flip(image, 0)

    # Update bounding box coordinates
    flipped_annotations = []
    for annotation in annotations:
        label, x_center, y_center, width, height = annotation
        # Adjust the y-coordinate to reflect the vertical flip
        y_center = 1 - y_center
        flipped_annotations.append((label, x_center, y_center, width, height))
    
    return flipped_image, flipped_annotations

In [ ]:
def scale_image(image):
    scale = random.uniform(0.5, 1.5)  
    # Get image dimensions
    (h, w) = image.shape[:2]

    # Perform the scaling
    scaled_image = cv2.resize(image, (int(w*scale), int(h*scale)), interpolation = cv2.INTER_LINEAR)

    return scaled_image, scale

def scale_annotations(annotations, scale):
    scaled_annotations = []
    for annotation in annotations:
        label, x_center, y_center, width, height = annotation
        # Adjust the coordinates to reflect the scaling
        x_center *= scale
        y_center *= scale
        width *= scale
        height *= scale
        scaled_annotations.append((label, x_center, y_center, width, height))

    return scaled_annotations


In [ ]:
def color_jitter(image, annotations):
    # Randomly adjust brightness, contrast, saturation, and hue
    alpha = np.random.uniform(0.5, 1.5)  # Adjust brightness (0.8 to 1.2)
    beta = np.random.uniform(0.8, 1.2)   # Adjust contrast (0.8 to 1.2)
    gamma = np.random.uniform(0.8, 1.2)  # Adjust saturation (0.8 to 1.2)
    delta = np.random.uniform(-10, 10)   # Adjust hue (-10 to 10)

    # Convert to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Adjust brightness
    hsv_image[:, :, 2] = np.clip(alpha * hsv_image[:, :, 2], 0, 255)

    # Adjust contrast
    image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
    image = np.clip(beta * image, 0, 255).astype(np.uint8)

    # Adjust saturation
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_image[:, :, 1] = np.clip(gamma * hsv_image[:, :, 1], 0, 255)

    # Adjust hue
    hsv_image[:, :, 0] = (hsv_image[:, :, 0] + delta) % 180

    image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

    return image, annotations

In [ ]:
def apply_blur(image, annotations):
    # Randomly select the blur kernel size
    kernel_size = np.random.choice([25, 15, 19, 29])  # You can customize the kernel sizes

    # Apply Gaussian blur to the image
    blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

    return blurred_image, annotations

In [ ]:
def apply_noise(image, annotations):
    # Generate random Gaussian noise
    noise = np.random.normal(0, 1, image.shape).astype(np.uint8)

    # Add noise to the image
    noisy_image = cv2.add(image, noise)

    return noisy_image, annotations

In [ ]:
def crop_image(image, annotations):


    
    cropped_annotations = []
    label, x_center, y_center, width, height = annotations[0]
    y_start = random.uniform(0, y_center)
    x_start = random.uniform(0, x_center)
    x_end = random.uniform(x_center, 1)
    y_end = random.uniform(y_center, 1)    
    
    cropped_image = image[int(y_start*image.shape[0]):int(y_end*image.shape[0]), int(x_start*image.shape[1]):int(x_end*image.shape[1])]
    y_center = (y_center - y_start) / (y_end - y_start)
    x_center = (x_center - x_start) / (x_end - x_start)
    cropped_annotations.append((label, x_center, y_center, width, height))
    return cropped_image, annotations



In [ ]:
def color_transform(image, annotations):
# Get the height and width of the image
    height, width, _ = image.shape

# Create a random color transformation matrix
    random_color_matrix = np.random.rand(3, 3)/3  # A 3x3 matrix with random values between 0 and 1

    transformed_image = np.dot(image.copy().reshape(-1, 3), random_color_matrix.T).reshape(height, width, 3)
    transformed_image = np.clip(transformed_image, 0, 255).astype(np.uint8)  # Clip and convert to uint8


    return transformed_image, annotations


In [ ]:
def rotate(image, annotations):
    rotation_center = (image.shape[1] // 2, image.shape[0] // 2)  # Center of the image

    # Define the rotation angle in degrees
    rotation_angle = np.random.randint(0, 90)  # Replace with your desired rotation angle

    # Calculate the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(rotation_center, rotation_angle, 1.0)

    # Rotate the entire image
    rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
    rotated_annotations = []
    for annotation in annotations:
        label, x_center, y_center, width, height = annotation
        x_center *= image.shape[1]
        y_center *= image.shape[0]
        width *= image.shape[1]
        height *= image.shape[0]
        if width>= height:
            diagonal = width
        else: diagonal = height
        # diagonal = np.sqrt(height**2 + width**2)
        point_to_locate = (x_center, y_center) 

        theta = np.rad2deg(np.arctan(height/width)) + rotation_angle
        point_after_rotation = np.dot(rotation_matrix, np.array([point_to_locate[0], point_to_locate[1], 1]))
        width_after_rotation = diagonal*np.abs(np.cos(np.deg2rad(theta)))/image.shape[1]
        height_after_rotation = diagonal*np.abs(np.sin(np.deg2rad(theta)))/image.shape[0]
        x_after_rotation = int(point_after_rotation[0])/image.shape[1]
        y_after_rotation = int(point_after_rotation[1])/image.shape[0]
        rotated_annotations.append((int(label), x_after_rotation, y_after_rotation, width_after_rotation, height_after_rotation))
        print(rotated_annotations)
    return rotated_image, rotated_annotations

In [ ]:
def image_comparison(image, annotations, sub_image, sub_annotations):
    print(sub_annotations)            
    draw_bounding_boxes(image, annotations)
    draw_bounding_boxes(sub_image, sub_annotations)        
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(sub_image, cv2.COLOR_BGR2RGB))
    plt.title("Transformed Image")
    plt.show()

In [ ]:
def apply_transformation(image, annotations):
    choices = np.array([1, 2, 3, 4, 5, 6, 7, 8])

# Define the probabilities for each integer
    probabilities = np.array([0.135, 0.135, 0.135, 0.135, 0.135, 0.135, 0.135, 0.055])

# Number of random integers to generate
    num_samples = 4

# Generate an array of random integers based on the specified probabilities
    # order = np.random.choice(choices, size=num_samples, p=probabilities)
    order = np.random.randint(1,9, 3)

    print(order)
    sub_image, sub_annotations = image, annotations
    for i in order:
        if i == 1:
            print('h_flip')
            sub_image, sub_annotations = horizontal_flip(
                sub_image, sub_annotations)

            # image_comparison(image, annotations, sub_image, sub_annotations)

        elif i == 2:
            print('v_flip')
            sub_image, sub_annotations = vertical_flip(
                sub_image, sub_annotations)

            # image_comparison(image, annotations, sub_image, sub_annotations)

        elif i == 3:
            print('color_jitter')
            sub_image, sub_annotations = color_jitter(
                sub_image, sub_annotations)

            # image_comparison(image, annotations, sub_image, sub_annotations)

        elif i == 4:
            print('apply_blur')
            sub_image, sub_annotations = apply_blur(sub_image, sub_annotations)

            # image_comparison(image, annotations, sub_image, sub_annotations)

        elif i == 5:
            print('noise')
            sub_image, sub_annotations = apply_noise(
                sub_image, sub_annotations)

            # image_comparison(image, annotations, sub_image, sub_annotations)

        elif i == 6:
            print('scale')
            sub_image, sub_scale = scale_image(sub_image)

            # image_comparison(image, annotations, sub_image, sub_annotations)
        elif i == 7:
            print('color_transform')
            sub_image, sub_annotations = color_transform(
                sub_image, sub_annotations)
        elif i == 8:
            print('rotate')
            sub_image, sub_annotations = rotate(sub_image, sub_annotations)
        elif i==9:
            print('crop')
            sub_image, sub_annotations = crop_image(sub_image, sub_annotations)

    image_comparison(image, annotations, sub_image, sub_annotations)
    
    return order, sub_image, sub_annotations
    


In [ ]:
def create_label_file(label_destination, original_name,annotations, i,dataset_size):
    file_path = os.path.join(label_destination, f'{dataset_size+i+dataset_size}_{original_name}.txt')
    with open(file_path, "w") as file:
        for annotation in annotations:
            content = ""
            for i, item in enumerate(annotation):
                if i == 0:
                    content += str(int(item))  # Ensure the first item is an integer
                else:
                    content += str(float(item))  # Convert the rest to floats
                if i < len(annotation) - 1:
                    content += " " 
            file.write(content + "\n")  
    # File is automatically closed when the 'with' block exits
    print(f"Content has been written to {file_path}")


In [ ]:
def create_image_file(image_destination, original_name, image, i, dataset_size):
    image_path = os.path.join(image_destination, f'{dataset_size+i+dataset_size}_{original_name}.jpg')
    cv2.imwrite(image_path, image)


In [ ]:
image_source = 'Initial_dataset\images'
label_source = 'Initial_dataset\labels'

image_destination  = 'transformed_dataset\images'
label_destination  = 'transformed_dataset\labels'

In [ ]:
def generate_dataset(image_source, label_source, image_destination, label_destination, n):
    
    dataset_size = len(os.listdir(image_source))
    for i in range(n):
        if i>=dataset_size:
            i = i-dataset_size
        print(os.listdir(image_source))
        image_name = os.listdir(image_source)[i]
        image_path = os.path.join(image_source, image_name)
        print(image_name, image_path)
        image = cv2.imread(image_path)
        label_path = os.path.join(label_source, os.path.splitext(image_name)[0]+'.txt')
        print(label_path)
        with open(label_path) as f:
            annotations = [tuple(map(float, line.split())) for line in f]
        order, image, annotations = apply_transformation(image, annotations)
        create_image_file(image_destination, os.path.splitext(image_name)[0], image, i,dataset_size)
        create_label_file(label_destination, os.path.splitext(image_name)[0], annotations, i,  dataset_size)


In [ ]:
generate_dataset(image_source, label_source, image_destination, label_destination, 3000)